In [2]:
#!rm -r /home/jovyan/.local/share/jupyter/kernels/nilmtk
!mamba install tensorflow-gpu==2.10 -y -q
# !mamba install tensorflow-gpu==2.11.0 -y -q
!pip uninstall -y -q nilmtk nilm_metadata
#!pip install -q pandas networkx tables scikit-learn hmmlearn pyyaml matplotlib==3.1.3 xgboost pyts
# Trick to install NILM regardless of its dependencies
#!pip install --no-cache -U git+https://github.com/nilmtk/nilm_metadata.git
#!pip install --no-cache -U git+https://github.com/nilmtk/nilmtk.git@0.4.3
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilmtk@master
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilm_metadata@master

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... By downloading and using the CUDA Toolkit conda packages, you accept the terms and conditions of the CUDA End User License Agreement (EULA): https://docs.nvidia.com/cuda/eula/index.html

By downloading and using the cuDNN conda packages, you accept the terms and conditions of the NVIDIA cuDNN EULA -
  https://docs.nvidia.com/deeplearning/cudnn/sla/index.html

done
  Cloning https://github.com/nilmtk/nilmtk (to revision master) to /tmp/pip-req-build-nhisx402
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilmtk /tmp/pip-req-build-nhisx402
  Resolved https://github.com/nilmtk/nilmtk to commit b2c514479cef478cab872cb635056da08d5352a1
  Preparing metadata (setup.py) ... done
  Created wheel for nilmtk: filename=nilmtk-0.4.0.dev1+git.b2c5144-py3-none-any.whl size=279177 sha256=7548499459516df5fececd9d2c607ef298afc414c183b34778c02bcb931728

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# from nilmtk import DataSet
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from joblib import Memory
import pickle
from pprint import pprint

#from nilmtk import DataSet
import multiprocessing as mp

from typing import Dict

!pip install -q scikit-learn-intelex
#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn import pipeline, metrics, linear_model, model_selection, multioutput, tree, ensemble, neural_network
!pip install xgboost -q
import xgboost as xgb

import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler

import numpy as np
#import graphviz
import keras
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
#from ann_visualizer.visualize import ann_viz
from matplotlib import pyplot as plt 
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix
# GPU goes brrrrrrrrrrrrrrrrrrrr
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

!pip3 install tqdm -q
from tqdm.notebook import tqdm
import NUK

# import garbage collector
import gc
import time


physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print(physical_devices)
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("No GPU found, model will train on CPU.")


2023-11-17 08:45:25.289376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100 80GB PCIe MIG 3g.40gb, compute capability 8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Preprocessing

# Models

In [5]:
# Parameters

epochs =1200
window_size = 2688
batch_size = 512
NmDevices = 64
# table_of_options = [1,2,3,4]
k = 0.125
# Define a learning rate scheduler function
def scheduler(epoch, lr):
    return lr
    if epoch == 0 or epoch == 1:
        return lr
    if epoch == 35:
        return lr *0.5
    else:
        return lr

RANDOM_SEED = 42
test_size = 0.2
lambda_l2=0
function = "GRU"

In [6]:
data= pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2688_numW_100000_upper32_gap3600_numD64_ideal.pkl")
# data = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2688_numW_50000_upper32_gap3600_numD64_ideal.pkl") 
# data_syn_ideal = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2688_numW_50000_upper32_gap3600_numD64_ideal.pkl") 
# data_syn_unmetered = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2688_numW_50000_upper32_gap3600_numD64_unmetered.pkl")

labels = pd.read_pickle("../../Energy_graph/data/labels_new.pkl")


# Separate the tuples into X and y
# X_syn_ideal = np.array([i[0] for i in data_syn_ideal])
# y_syn_ideal = np.array([i[1] for i in data_syn_ideal])

# X_syn_unmetered = np.array([i[0] for i in data_syn_unmetered])
# y_syn_unmetered = np.array([i[1] for i in data_syn_unmetered])

X = np.array([i[0] for i in data])
y = np.array([i[1] for i in data])


# X = np.concatenate((X, X_syn_ideal,X_syn_unmetered), axis=0)
# y = np.concatenate((y, y_syn_ideal, y_syn_unmetered), axis=0)


y = y.astype(int)


# X_test = np.array([i[0] for i in data_test])
# y_test = np.array([i[1] for i in data_test])


In [7]:

def normalize(X):
    max_value = 0

    for x in X:
        v = np.max(x)
        if v > max_value:
            max_value = v

    if max_value == 0:
        return X
    return X / max_value

    




In [5]:
len(labels)

64

## InceptionTime

### InceptionTime class

In [8]:
# data2= pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2688_numW_100000_upper32_gap3600_numD64_ideal.pkl")
# y2 = np.array([i[1] for i in data])
class_weighs_pre = NUK.class_weights_tool(y)

class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, iteration, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500, lr=0.001):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.iteration = iteration

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            # if (verbose == True):
                # self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='sigmoid')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=self.lr),
                      metrics=['accuracy', NUK.F1Score, NUK.WeightedF1Score(class_weighs_pre)])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.25, patience=7, min_lr=0.0001)
        early_stopping = keras.callbacks.EarlyStopping(patience=9, restore_best_weights=True, monitor='loss')

        file_path = self.output_directory + f'best_model_{self.iteration}.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)
        # lr_scheduler = LearningRateScheduler(scheduler)
        self.callbacks = [reduce_lr, model_checkpoint, early_stopping]

        return model

    def fit(self, x_train, y_train):
       
        # x_val and y_val are only used to monitor the test loss and NOT for training

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, callbacks=self.callbacks, validation_split=0.1)


        self.model.save(self.output_directory + f'last_model_{self.iteration}.hdf5')

        keras.backend.clear_session()

        return None

    def predict(self, x_test):
        model_path = self.output_directory + f'last_model_{self.iteration}.hdf5'
        model = keras.models.load_model(model_path, custom_objects={'F1Score': NUK.F1Score, "WeightedF1": NUK.WeightedF1Score(class_weighs_pre)})
        y_pred = model.predict(x_test, batch_size=self.batch_size)
    #     # if return_df_metrics:
    #     #     y_pred = np.argmax(y_pred, axis=1)
    #     #     df_metrics = calculate_metrics(y_true, y_pred, 0.0)
    #     #     return df_metrics
    #     # else:
    #     #     test_duration = time.time() - start_time
    #     #     save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
    #     #     return y_pred
        return y_pred


### Single model

In [ ]:

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
itr = 0
for train_index, test_index in tqdm(kf.split(X)):
    # print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train = normalize(X_train)
    X_test = normalize(X_test)
    # model = NUK.PC0_reg(NmDevices, window_size, function, 128, k, lambda_l2=lambda_l2)
    print(batch_size, epochs)
    model = Classifier_INCEPTION(output_directory="./models/", input_shape=(window_size, 1), nb_classes=78, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=6)
    model.fit(X_train, y_train)
    # model.model.predict(X_test, batch_size=batch_size)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > 0.5)

    predictions.append((y_pred_tf, y_test, y_pred))
    t = (y_pred_tf, y_test, y_pred)
    with open(f"./predictions/test_inception_pickle_{itr}.pkl", 'wb') as f:
        pickle.dump(t, f)

    itr += 1
    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/Inception_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


### Ensemble

#### k-fold

In [31]:
num_splits = 3
kf = KFold(n_splits=num_splits, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
for fold,(train_index, test_index) in tqdm(enumerate(kf.split(X))):
    # print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # normalization
    X_train = normalize(X_train)
    X_test = normalize(X_test)
    model_predictions = []

    print(batch_size, epochs)
    for i in range(5):

        model = Classifier_INCEPTION(output_directory=f"./models/{fold}/", input_shape=(window_size, 1), nb_classes=78, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=8, iteration=i)
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        model_predictions.append(y_pred)

    y_pred = np.mean(model_predictions, axis=0)
    y_pred_tf = (y_pred > 0.5)

    predictions.append((y_pred_tf, y_test, y_pred))
    t = (y_pred_tf, y_test, y_pred)
    with open(f"./predictions/test_inception_pickle_{fold}.pkl", 'wb') as f:
        pickle.dump(t, f)

    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / num_splits
import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/NNE_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_3fold.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


KeyboardInterrupt: 

#### 80/20 split

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train = normalize(X_train)
X_test = normalize(X_test)

evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
model_predictions = []

print(batch_size, epochs)
for i in range(5):

    model = Classifier_INCEPTION(output_directory=f"./models/performance/", input_shape=(window_size, 1), nb_classes=NmDevices, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=10, iteration=i, kernel_size=128, kernel_size=128)
    # model = keras.models.load_model(f"./models/test/last_model_{i}.hdf5", custom_objects={'F1Score': NUK.F1Score, "WeightedF1": NUK.WeightedF1Score(class_weighs_pre)})
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    model_predictions.append(y_pred)

y_pred = np.mean(model_predictions, axis=0)
y_pred_tf = (y_pred > 0.3)

predictions.append((y_pred_tf, y_test, y_pred))
t = (y_pred_tf, y_test, y_pred)
with open(f"./predictions/test_inception_pickle_performance_testing.pkl", 'wb') as f:
    pickle.dump(t, f)

report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
print(report["weighted avg"])

evaluation_results.append(report)
gc.collect()
tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T

import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/NNE_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_performance_testing.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


### Performance testing

In [9]:
epochs = 5
window_size = 2688
batch_size = 512



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train = normalize(X_train)
X_test = normalize(X_test)

evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
model_predictions = []

print(batch_size, epochs)
for i in range(1):

    model = Classifier_INCEPTION(output_directory=f"./models/performance/", input_shape=(window_size, 1), nb_classes=NmDevices, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=10, iteration=i, kernel_size=128)
    # model = keras.models.load_model(f"./models/test/last_model_{i}.hdf5", custom_objects={'F1Score': NUK.F1Score, "WeightedF1": NUK.WeightedF1Score(class_weighs_pre)})
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    model_predictions.append(y_pred)

y_pred = np.mean(model_predictions, axis=0)
y_pred_tf = (y_pred > 0.3)

predictions.append((y_pred_tf, y_test, y_pred))
t = (y_pred_tf, y_test, y_pred)
with open(f"./predictions/test_inception_pickle_performance_testing.pkl", 'wb') as f:
    pickle.dump(t, f)

report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
print(report["weighted avg"])

evaluation_results.append(report)
gc.collect()
tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T

import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/NNE_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_performance_testing.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


512 5
Epoch 1/5
207/207 [==============================] - 330s 2s/step - loss: 0.3249 - accuracy: 0.0859 - F1Score: 0.0203 - WeightedF1: 0.1213 - val_loss: 0.3521 - val_accuracy: 5.9544e-04 - val_F1Score: 0.0026 - val_WeightedF1: 0.0015 - lr: 0.0010
Epoch 2/5
207/207 [==============================] - 286s 1s/step - loss: 0.2847 - accuracy: 0.1037 - F1Score: 0.0501 - WeightedF1: 0.2598 - val_loss: 0.3316 - val_accuracy: 0.0115 - val_F1Score: 0.0057 - val_WeightedF1: 0.0091 - lr: 0.0010
Epoch 3/5
207/207 [==============================] - 286s 1s/step - loss: 0.2765 - accuracy: 0.1190 - F1Score: 0.0766 - WeightedF1: 0.3217 - val_loss: 0.3225 - val_accuracy: 0.0249 - val_F1Score: 0.0224 - val_WeightedF1: 0.0388 - lr: 0.0010
Epoch 4/5
207/207 [==============================] - 286s 1s/step - loss: 0.2699 - accuracy: 0.1256 - F1Score: 0.1004 - WeightedF1: 0.3611 - val_loss: 0.2979 - val_accuracy: 0.0947 - val_F1Score: 0.1027 - val_WeightedF1: 0.2987 - lr: 0.0010
Epoch 5/5
58/58 [=========

Closing remaining open files:/tmp/nilmtk-jgsyvjt5.h5...done


## ResNet

In [20]:
def build_resnet(input_shape, n_feature_maps, nb_classes):
    print ('build conv_x')
    x = keras.layers.Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv1D(n_feature_maps, 8, 1, padding='same')(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps, 1, 1,padding='same')(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv1D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv1D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    full = keras.layers.GlobalAveragePooling1D()(y)
    out = keras.layers.Dense(nb_classes, activation='sigmoid')(full)
    print ('        -- model was built.')
    # input = keras.layers.Input(shape=(input_shape))
    model = keras.models.Model(inputs=x, outputs=out, name="Resnet")

    return model

In [ ]:
num_splits = 5
kf = KFold(n_splits=num_splits, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
treshold = 0.5
debug = 0
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train[:, 0] = normalize(X_train[:, 0])
    X_test[:,0] = normalize(X_test[:, 0])

    model = build_resnet((window_size, 1), 32, NmDevices)
    # model.build((len(y_train) + len(y_test), window_size, 1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.25, patience=3, min_lr=0.000001)
    early_stopping = keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True, monitor='loss')

    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[reduce_lr,early_stopping])

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > treshold)

    predictions.append((y_pred, y_pred_tf, y_test))



    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"], "|||", report["samples avg"])

    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T
df = df / num_splits
import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/ResNet_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_mixed_t0.3_d10.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")




    



## PC0

In [ ]:

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
treshold = 0.5
debug = 0
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train[:, 0] = normalize(X_train[:, 0])
    X_test[:,0] = normalize(X_test[:, 0])

    # break
    # class_weights_all = NUK.class_weights_tool(y_train)
    # for k in class_weights_all:
    #     class_weights_all[k] += 1
    
    # print(class_weights_all)
    # break
    model = NUK.PC0_reg(NmDevices, window_size, function, 128, k, lambda_l2=lambda_l2)
    # model = NUK.VGG11_1D(NmDevices, window_size)
    model.build((len(y_train) + len(y_test), window_size, 1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
    # lr_scheduler = LearningRateScheduler(scheduler)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.25, patience=3, min_lr=0.000001)
    # callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1,patience=2, min_lr=0.0000002)
    # for class weights
    # model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weights_UKD, callbacks=[lr_scheduler])
    # class_weights = NUK.class_weights_tool(y_train)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[reduce_lr])

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > treshold)

    predictions.append((y_pred, y_pred_tf, y_test))

    # n_labels = y_test.shape[1]
    # co_occurrence_matrix = np.zeros((n_labels, n_labels))

    # for true, pred in zip(y_test, y_pred_tf):
    #     fn_labels = np.where((true == 1) & (pred == 0))[0]  # False negatives
    #     fp_labels = np.where((true == 0) & (pred == 1))[0]  # False positives

    #     for fn in fn_labels:
    #         for fp in fp_labels:
    #             co_occurrence_matrix[fn, fp] += 1



    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"], "|||", report["samples avg"])

    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()



    


## TSMixer
https://github.com/google-research/google-research/tree/master/tsmixer/tsmixer_basic/models

In [ ]:
#TODO implement TSMixer for multilabel classification

## Random forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
jobs = 110
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []

for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Initialize the Random Forest classifier wrapped in a MultiOutputClassifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, n_jobs=jobs)
    # classifier = MultiOutputClassifier(forest, n_jobs=-1)
    classifier.fit(X_train, y_train)

    # Predict the labels
    y_pred = classifier.predict(X_test)

    # Evaluate the model using classification report
    report = classification_report(y_test, y_pred, target_names=labels, zero_division=0, output_dict=True)

    evaluation_results.append(report)


for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
# df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_staticLR_LSTM_watts_mixed100k.csv")
df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_classes{NmDevices}_watts_RF_mixed50k.csv")
    


0it [00:00, ?it/s]

Train indices: [     1      2      3 ... 147053 147054 147055] Test indices: [     0      4     12 ... 147049 147050 147052]


In [ ]:
evaluation_results = []
model = NUK.PC0(NmDevices, window_size, 'GRU',128, k)
model.build((len(y), window_size, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
lr_scheduler = LearningRateScheduler(scheduler)

model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=[lr_scheduler])


y_pred = model.predict(X_test)
y_pred_tf = (y_pred > 0.5)
report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)

# report_DF = NUK.ClassificationReportToDF(report, labels)  
# report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
df = pd.DataFrame(report)

## CatBoost

In [4]:
%pip install catboost
import catboost as cb


Note: you may need to restart the kernel to use updated packages.


In [15]:


kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
debug = 0
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train = normalize(X_train)
    X_test = normalize(X_test)

    # break

    # Initialize CatBoost multilabel classifier
    model = cb.CatBoostClassifier(
        loss_function='MultiLogloss',
        verbose=500,
        random_seed=RANDOM_SEED,
        task_type="GPU",
        devices='0:1',
        # class_weights=class_weights_all,
        iterations=500,

        )  # Add other hyperparameters if needed

    # Train the model
    model.fit(X_train, y_train)

    # Get predictions
    y_pred = model.predict(X_test)

    # Convert predictions to boolean format similar to y_test for evaluation
    y_pred_tf = (y_pred == 1)
    print(y_pred)

    predictions.append((y_pred_tf, y_test))




    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    break
    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    del model



    


0it [00:00, ?it/s]

Train indices: [    0     1     2 ... 49997 49998 49999] Test indices: [    4     6     7 ... 49988 49990 49992]
Learning rate set to 0.051213
0:	learn: 0.6478853	total: 545ms	remaining: 4m 32s
499:	learn: 0.2732736	total: 3m 57s	remaining: 0us
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'precision': 0.12109148155906953, 'recall': 0.021307596513075965, 'f1-score': 0.03282443454939719, 'support': 80300}


In [16]:
y_pred[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [57]:
gc.collect()
tf.keras.backend.clear_session()


## Evaluation

In [12]:
import os
for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/PC0_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_mix.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")
# df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_classes{NmDevices}_watts_RF_mixed50k.csv")
    
# evaluation_results+ evaluation_results[1].to_

In [30]:
model = tf.keras.models.load_model("./models/test/last_model_4.hdf5", custom_objects={'F1Score': NUK.F1Score, "WeightedF1": NUK.WeightedF1Score(class_weighs_pre)})
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2688, 1)]    0           []                               
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 2688, 1)      0           ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 2688, 32)     1280        ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 2688, 32)     640         ['input_1[0][0]']                
                                                                                              

In [23]:
df = pd.read_pickle("../../Energy_graph/data/processed_watts/REFIT_clean.pkl")
df["REFIT_1"]

import pickle

with open("../../Energy_graph/data/processed_watts/refit/REFIT1_clean.pkl", 'wb') as f:
    pickle.dump({"REFIT_1" : df["REFIT_1"]}, f)

In [9]:
import pandas as pd
import os
import re
from tqdm import tqdm
import pickle
from collections import Counter
import concurrent.futures
import argparse
# from helper import preprocess_string

import re

def preprocess_string(string : str) -> str:
    string = string.lower().strip()
    string = re.sub(' +', ' ', string)
    string = string.replace("_", " ")
    string = string.replace("-", " ")
    string = string.replace("&", " ")
    string = string.split("(")[0]
    string = string.split("#")[0]

    string = string.strip()

    # handle known synoynms
    synonyms = {
        "refrigerator": "fridge",
        "vaccumcleaner": "vacuum cleaner",
        "breadmaker": "bread maker",
      
        
    }
    if "freezer" in string:
        string = "fridge"

    if string in synonyms:
        string = synonyms[string]

    if 'hi fi' in string:
        string = "audio system"

    if "router" in string:
        string = "router"

    if "treadmill" in string:
        string = "running machine"
        

    if "laptop" in string:
        string = "laptop"
    
    if "server" in string:
        string = "server"

    if "monitor" in string and not "baby" in string:
        string = "monitor"
    # special cases
    if "computer" in string and "charger" not in string:
        string = "pc"

    if "tv" in string:
        string = "television"

    if "television" in string:
        string = "television"

    if "macbook" in string:
        string = "laptop"
        
    if "car charger" == string:
        string = "ev"
    
    if "toast" in string:
        string = "toaster"
    
    if "modem" in string:
        string = "router"

    # we treat all audio devices as speakers so subwoofer is also a speaker
    if "subwoofer" in string:
        string = "speaker"

    if "speaker" in string:
        string = "speaker"

    if "iron" in string and "soldering" not in string:
        string = "iron"

    
    if "coffeemachine" in string:
        string = "coffee machine"
    if "coffee maker" in string:
        string = "coffee machine"

    if "dishwasher" in string:
        string = "dish washer"
    if "air conditioner" in string:
        string = "ac"

    if "air conditioning" in string:
        string = "ac"
    
    string = re.sub(' +', ' ', string)
    string = re.sub(r'\d+', '', string)
    return string.strip()





def process_dictionary(data: dict, values=0) -> pd.DataFrame:

    ignored_devices = [
        "light",
        "outlet",
        "sockets",
        "lamp",
        "plug",
        'CE appliance'
        'kettle/toaster',
        'dehumidifier/heater',
        'HairDryer-Straightener',
        'Office Desk',
        'heat basement',
        'set top box',
        'subpanel',
    ]
    dfs = []
    for device in data:
        # ignore devices
        if any(ignored_device in device.lower() for ignored_device in ignored_devices):
            continue
        if device == "aggregate":
            continue
        # preprocess device name
        device_name = preprocess_string(device)
        
        df = data[device]
        df = df.resample("8s").mean()

        # rename column to standardized device name
        df.columns = [device_name]
        if df.max().max() < 2:
            print("device with zeros: ", device_name)
            continue

        time_diffs = df.index.to_series().diff()
        median_interval = time_diffs.median()

        # if there is less than 3 days of data drop the device
        if len(df) < (3*24 * 60 * 60) / median_interval.total_seconds():
            print("less than 3 days of data for device: ", device_name)
            continue
        df.dropna(inplace=True)
        dfs.append(df)

    # concatenate all dataframes
    df = pd.concat(dfs, axis=1)

    # handle missing values
    df = df.ffill(limit=6)
    df.fillna(0, inplace=True)

    # handle negative values
    df[df<0] = 0

    df["aggregate"] = df.sum(axis=1)
    # df.drop(columns=["aggregate"], inplace=True)

    # df.rename(columns={"sum_ideal": "aggregate"}, inplace=True)



    # treshold in watts
    treshold = 5
    if values == 0:
        # put 1 if device is on and 0 if device is off
        for c in df.columns:
            if c == "aggregate":
                continue
            # if power is less than treshold device is off
            df[c] = (df[c] > treshold).astype(int)

    # find duplicate columns
    column_counts = Counter(df.columns)
    duplicates = [col for col, count in column_counts.items() if count > 1]
    # Sum duplicate columns
    for duplicate in duplicates:
        duplicate_cols = [col for i, col in enumerate(df.columns) if col == duplicate]
        df[duplicate] = df[duplicate_cols].sum(axis=1)
        # Drop other duplicate columns if needed
        df = df.loc[:, ~df.columns.duplicated(keep='last')]
    
    
    return df
    


def process_dataset(dataset_path):
    data = pd.read_pickle(dataset_path)
    # print(dataset_path)
    for house in data:
        data[house] = process_dictionary(data[house], house)
        
    return data


def get_data(path : str, labels_path : str, values=0):
        
    # path = "./Energy_graph/data/processed_watts/"
    dataset_paths = [os.path.join(path, dataset) for dataset in os.listdir(path) if dataset.endswith('.pkl')]
        
    cpu_count = int(os.cpu_count() / 2)
    data_dict = {}

    with tqdm(total=len(dataset_paths), desc="Processing datasets", unit="dataset") as progress_bar:
        with concurrent.futures.ProcessPoolExecutor(max_workers=cpu_count) as executor:
            futures = {executor.submit(process_dataset, dataset_path): dataset_path for dataset_path in dataset_paths}
            
            for future in concurrent.futures.as_completed(futures):
                dataset_path = futures[future]
                try:
                    processed_data = future.result()
                    data_dict.update(processed_data)
                except Exception as e:
                    print(f"Dataset {dataset_path} generated an exception: {e}")
                
                progress_bar.update(1)

    labels = pd.read_pickle(labels_path)
    labels.sort()

    return data_dict

def create_windows(data : dict, labels_path: str, save_path : str, time_window=2700, upper_bound=pd.Timedelta(seconds=32), max_gap = pd.Timedelta(seconds=3600)):
    """Creates windows of time_window seconds from the data and discards windows with gaps of more than 1h or 15 gaps of 32 seconds or more"""
    
    labels = pd.read_pickle(labels_path)
    # windows = []
    X_Y = [] # list of tuples (X, Y)
    skip_count_1 = 0
    skip_count_2 = 0
    skip_count_3 = 0
    total_count = 0

    for df in data.values():
        print(len(df["aggregate"]))
        for i in range(0, len(df) - time_window, time_window + 1):
            window = df.iloc[i:i + time_window]
            total_count += 1
            # if there is a gap of more than max_gap skip the window
            time_diffs = window.index.to_series().diff().dropna()
            if  (time_diffs >= max_gap).any():
                skip_count_1 += 1
                continue
            # if there are more than 15 gaps of upper_bound or more skip the window
            if len(time_diffs[time_diffs > upper_bound]) > 15:
                skip_count_2 += 1
                continue

            x = window["aggregate"].values
            # if there is a value bigger than 50000 skip the window
            if (x > 50000).any():
                skip_count_3 += 1
                continue
            devices = [False] * len(labels)
            # check if device is on in the window
            for c in window.columns:
                if c == "aggregate":
                    continue
                on = (window[c] > 0)
                ix = labels.index(c)
                devices[ix] = on.any()

            X_Y.append((x, devices))
            


            # windows.append(window)
    print("Total windows: ", total_count, "Skipped windows due to 30min gap: ", skip_count_1, "Skipped windows due to 15 gaps of 32s or more: ", skip_count_2 ,"Skipped windows due to values larger than 50k: ", skip_count_3 ,"Procentage skipped: ", (skip_count_1+skip_count_2+ skip_count_3) / total_count * 100)
    return X_Y
    # with open(save_path+ f"/X_Y_wsize{time_window}_upper{int(upper_bound.total_seconds())}_gap{int(max_gap.total_seconds())}_numD{len(labels)}.pkl", "wb") as f:
    #     pickle.dump(X_Y, f, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:

folder = ""
# Initialize paths
path_to_base = "../../Energy_graph/"
path = path_to_base + "/data/processed_watts/"+folder
labels_path = path_to_base + "/data/labels_new.pkl"
save_path = path_to_base + "/data/training_data/processed/"+folder

# Initialize parameters
time_window = 2688
upper_bound = pd.Timedelta(seconds=32)
max_gap = pd.Timedelta(seconds=3600)

# Print parameters
print("Path to base folder: ", path_to_base)
print("Path to save windows: ", save_path)
print("Time window: ", time_window, "rows | ", time_window*8, "seconds")
print("Upper bound: ", upper_bound)
print("Max gap: ", max_gap)

# Get data and create windows
data = get_data(path, labels_path)
# create_windows(data, labels_path, save_path, time_window, upper_bound, max_gap)


Path to base folder:  ../../Energy_graph/
Path to save windows:  ../../Energy_graph//data/training_data/processed/
Time window:  2688 rows |  21504 seconds
Upper bound:  0 days 00:00:32
Max gap:  0 days 01:00:00


Processing datasets:   0%|          | 0/10 [00:00<?, ?dataset/s]

Processing datasets:  50%|█████     | 5/10 [00:06<00:04,  1.04dataset/s]

less than 3 days of data for device:  games console


Processing datasets: 100%|██████████| 10/10 [02:10<00:00, 13.07s/dataset]


In [11]:
# data.keys()
processed = {}

for key in tqdm(data.keys()):
    # if "UKDALE" not in key: 
    #     continue
    print(key, len(data[key]))
    processed[key] = create_windows({key : data[key]}, labels_path, save_path, time_window, upper_bound, max_gap)


  0%|          | 0/64 [00:00<?, ?it/s]

IAWE_1 620447
620447


  2%|▏         | 1/64 [00:00<00:20,  3.07it/s]

Total windows:  230 Skipped windows due to 30min gap:  14 Skipped windows due to 15 gaps of 32s or more:  0 Skipped windows due to values larger than 50k:  0 Procentage skipped:  6.086956521739131
HES_1 1152268
1152268


  5%|▍         | 3/64 [00:01<00:24,  2.50it/s]

...

Total windows:  530 Skipped windows due to 30min gap:  2 Skipped windows due to 15 gaps of 32s or more:  0 Skipped windows due to values larger than 50k:  0 Procentage skipped:  0.37735849056603776
UKDALE_2 2156879
2156879


 97%|█████████▋| 62/64 [01:13<00:04,  2.20s/it]

Total windows:  802 Skipped windows due to 30min gap:  8 Skipped windows due to 15 gaps of 32s or more:  0 Skipped windows due to values larger than 50k:  0 Procentage skipped:  0.997506234413965
UKDALE_1 17064863
17064863


100%|██████████| 64/64 [01:34<00:00,  1.48s/it]

Total windows:  6346 Skipped windows due to 30min gap:  35 Skipped windows due to 15 gaps of 32s or more:  0 Skipped windows due to values larger than 50k:  0 Procentage skipped:  0.5515285219035614
UKDALE_3 396227
396227
Total windows:  147 Skipped windows due to 30min gap:  7 Skipped windows due to 15 gaps of 32s or more:  0 Skipped windows due to values larger than 50k:  0 Procentage skipped:  4.761904761904762


In [6]:
data[key]

,aggregate,kettle,projector,laptop,electric space heater
0,,,,,
2013-02-27 20:35:12,5.0,0,0,0,0
2013-02-27 20:35:20,4.0,0,1,0,0
2013-02-27 20:35:28,5.0,0,1,0,0
2013-02-27 20:35:36,4.0,0,0,0,0
2013-02-27 20:35:44,4.0,0,0,0,0
...,...,...,...,...,...
2013-04-08 05:14:32,176.0,0,0,0,0
2013-04-08 05:14:40,174.0,0,0,0,0
2013-04-08 05:14:48,0.0,0,0,0,0


In [12]:
def get_devices(data):

    devices = set()
    real_str = " "
    for key in data.keys():
        if key == "aggregate":
            continue
        real_str+= key + ", "
        devices.add(preprocess_string(key))
        

    return list(devices), real_str

In [13]:
import pickle
with open("../../Energy_graph/data/training_data/real_house.pkl", "wb") as f:
    pickle.dump(processed, f, protocol=pickle.HIGHEST_PROTOCOL)
    

In [20]:
models = []
labels = pd.read_pickle(labels_path)

for f in os.listdir("./models/test_IDEAL/"):
        if "last" in f:
            model = tf.keras.models.load_model("./models/test_IDEAL/"+f, custom_objects={'F1Score': NUK.F1Score, "WeightedF1": NUK.WeightedF1Score(class_weighs_pre)})
            models.append(model)

for h in processed:
    print(h)
    df = processed[h]
    curr_devices, real_str = get_devices(data[h])
    X = np.array([i[0] for i in df])
    y = np.array([i[1] for i in df])
    X= normalize(X)
    print(len(X.shape))

    if len(X.shape) !=2 or X.shape[1] != 2688:
        print("wrong shape", X.shape)
        continue
    
    model_predictions = []
    for m in models:
            y_pred = m.predict(X)
            model_predictions.append(y_pred)


    y_pred_avg = np.mean(model_predictions, axis=0)
    y_pred_tf = np.where(y_pred_avg > 0.3, 1, 0)

    y_pred_num_devices = np.sum(y_pred_tf, axis=1)
    y_real_num_devices = np.sum(y, axis=1)

    # print("average number of devices misspredcited: ", np.mean(y_pred_num_devices - y_real_num_devices)	)

    res = metrics.classification_report(y, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(res["weighted avg"]["f1-score"], res["weighted avg"]["precision"], res["weighted avg"]["recall"])

    # barplot y_pred_tf
    plt.figure(figsize=(20,10))
    plt.bar(labels, y_pred_tf.sum(axis=0))
    # plt.title(h + real_str + "f1: "+ str(res["weighted avg"]["f1-score"]) + "precision: "+ str(res["weighted avg"]["precision"]) + "recall: "+ str(res["weighted avg"]["recall"]))
    plt.title(h + " f1: "+ str(res["weighted avg"]["f1-score"])[:5] + " average number of devices misspredcited: " + str(np.mean(y_pred_num_devices - y_real_num_devices))[:5]	)

    # Get current axis
    ax = plt.gca()

    # Loop through label list and set the color to red for highlighted labels
    for label in ax.get_xticklabels():
        if label.get_text() in curr_devices:
            label.set_color('red')

    # Rotate all labels
    plt.xticks(rotation=90)
    plt.savefig(f"./plots/real_ideal/{h}_barplot.svg", format="svg")
    plt.close()

IAWE_1
2
7/7 [==============================] - 1s 26ms/step
0.5008752495756639 0.6483688948976613 0.46766169154228854
HES_1
2
14/14 [==============================] - 0s 26ms/step
0.9336485220415739 1.0 0.8800236406619385
REDD_1
2
4/4 [==============================] - 0s 26ms/step
0.19309986010022545 0.4746600741656366 0.16563658838071693
REDD_2
2
2/2 [==============================] - 0s 25ms/step
0.48504702660240756 0.582089552238806 0.43656716417910446
REDD_3
2
2/2 [==============================] - 0s 25ms/step
0.19243143563985396 0.43884892086330934 0.1630695443645084
REDD_4
2
3/3 [==============================] - 0s 26ms/step
0.09473283976124885 0.37109375 0.05859375
REDD_5
2
1/1 [==============================] - 0s 19ms/step
0.4062575794324521 0.4838709677419355 0.3709677419354839
REDD_6
2
2/2 [==============================] - 0s 25ms/step
0.19418426691153964 0.3501683501683502 0.18518518518518517
DRED_1
2
20/20 [==============================] - 1s 26ms/step
0.361323039019

In [33]:
tb = pd.read_pickle("../../Energy_graph/data/tracebase/devices_data.pkl")
x = tb["fridge"][0][:2668]
x = x.values.reshape(1, -1)
# print(x.values.reshape(1, -1)[0])
model_predictions = []
for m in models:
    y_pred = m.predict(x)
    model_predictions.append(y_pred)
    
y_pred_avg = np.mean(model_predictions, axis=0)
y_pred_tf = np.where(y_pred_avg > 0.3, 1, 0)
print(y_pred_tf)
for i in range(len(labels)):
    if y_pred_tf[0][i] == 1:
        print(labels[i])
    

1/1 [==============================] - 0s 37ms/step
[[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0
  1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]
television
audio system
fridge
washer dryer
kettle
washing machine


In [14]:
import pandas as pd
import re
import os

def preprocess_string(string : str) -> str:
    string = string.lower().strip()
    string = re.sub(' +', ' ', string)
    string = string.replace("_", " ")
    string = string.replace("-", " ")
    string = string.replace("&", " ")
    string = string.split("(")[0]
    string = string.split("#")[0]

    string = string.strip()

    # handle known synoynms
    synonyms = {
        "refrigerator": "fridge",
        "vaccumcleaner": "vacuum cleaner",
        "breadmaker": "bread maker",
      
        
    }
    if "freezer" in string:
        string = "fridge"

    if string in synonyms:
        string = synonyms[string]

    if 'hi fi' in string:
        string = "audio system"

    if "router" in string:
        string = "router"

    if "treadmill" in string:
        string = "running machine"
        

    if "laptop" in string:
        string = "laptop"
    
    if "server" in string:
        string = "server"

    if "monitor" in string and not "baby" in string:
        string = "monitor"
    # special cases
    if "computer" in string and "charger" not in string:
        string = "pc"

    if "tv" in string:
        string = "television"

    if "television" in string:
        string = "television"

    if "macbook" in string:
        string = "laptop"
        
    if "car charger" == string:
        string = "ev"
    
    if "toast" in string:
        string = "toaster"
    
    if "modem" in string:
        string = "router"

    # we treat all audio devices as speakers so subwoofer is also a speaker
    if "subwoofer" in string:
        string = "speaker"

    if "speaker" in string:
        string = "speaker"

    if "iron" in string and "soldering" not in string:
        string = "iron"

    
    if "coffeemachine" in string:
        string = "coffee machine"
    if "coffee maker" in string:
        string = "coffee machine"

    if "dishwasher" in string:
        string = "dish washer"
    if "air conditioner" in string:
        string = "ac"

    if "air conditioning" in string:
        string = "ac"
    
    string = re.sub(' +', ' ', string)
    string = re.sub(r'\d+', '', string)
    return string.strip()

